# Existem muitas alternativas ao modelo imperativo do matplotlib

## "facilitações" em cima do modelo imperativo:

- pandas plotting
- hvplot (figuras interativas do modo fácil)
- bokeh (figuras interativas do modo difícil)

## modelo declarativo

- altair
- seaborn

## pandas

In [ ]:
from pathlib import Path
import pandas as pd

fname = Path('data', 'dados_pirata.csv')
pd.read_csv(fname).head()

In [ ]:
df = pd.read_csv(
    fname,
    index_col='datahora',
    na_values=-99999,
    parse_dates=True,
).drop('Unnamed: 0', axis=1)

df.tail()

In [ ]:
df.columns = [
    '{0:0>3}'.format(col.split('_')[1]) for col in df.columns
]

df.sort_index(axis=1, inplace=True)
df.head()

In [ ]:
desc = df.describe()
desc

In [ ]:
desc.loc['std'] ** 2

In [ ]:
%matplotlib inline


ax = df[['001', '100', '180', '500']].plot()

In [ ]:
df['001'].interpolate().plot()
df['001'].plot();

In [ ]:
df['001'].interpolate(method='time', limit=10).plot()
df['001'].plot();

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(9, 5))
r = df.resample(rule='M')
r.median().plot(ax=ax)

In [ ]:
key = lambda x: x.month

grouped = df.groupby(key)

monthly = grouped.mean()
monthly.shape

### hvplot

In [ ]:
import hvplot.pandas


monthly.hvplot()

### bokeh

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook


x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

p = figure(
    title="simple line example",
    x_axis_label='x',
    y_axis_label='y'
)

p.line(x, y, legend="Temp", line_width=2)

show(p)

### seaborn

In [ ]:
fname = Path('data', 'fish.csv')

df = pd.read_csv(
    fname,
    na_values='NaN',
    sep=',',
    skipinitialspace=True,
)

df.head()

In [ ]:
import seaborn


kw = {
    'axes.edgecolor': '0',
    'text.color': '0',
    'ytick.color': '0',
    'xtick.color': '0',
    'ytick.major.size': 5,
    'xtick.major.size': 5,
    'axes.labelcolor': '0'
}

seaborn.set_style('whitegrid', kw)

In [ ]:
corr = df.corr()
corr

In [ ]:
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

In [ ]:
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

cmap = seaborn.diverging_palette(220, 10, as_cmap=True)

seaborn.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=0.3,
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={'shrink': 0.5}
);

In [ ]:
seaborn.jointplot('Days', 'BDE 99 (ng/g)', df, kind='reg');

In [ ]:
seaborn.jointplot('Days', 'BDE 47 (ng/g)', df, kind='reg');

In [ ]:
seaborn.residplot('Days', 'BDE 47 (ng/g)', df);

### altair

In [ ]:
from vega_datasets import data


df = data.seattle_weather()
df.head()

In [ ]:
import altair


altair.renderers.enable('notebook')

altair.Chart(df).mark_tick().encode(
    x='precipitation',
).interactive()

In [ ]:
altair.Chart(df).mark_bar().encode(
    altair.X('precipitation', bin=True),
    y='count()'
).interactive()

In [ ]:
altair.Chart(df).mark_line().encode(
    x='month(date):T',
    y='average(precipitation)'
).interactive()

In [ ]:
altair.Chart(df).mark_bar().encode(
    x='month(date):N',
    y='count()',
    color='weather',
)

### windrose

In [ ]:
import pandas as pd


fname = Path('data', 'DD2018_DANIELAZANETTI3.xls')
df = pd.read_excel(
    fname,
    skiprows=10,
)

df.head(n=10)

In [ ]:
lat, lon = (
    df.iloc[1, 1].replace(',', '.'),
    df.iloc[2, 1].replace(',', '.'),
)

lat, lon = map(float, (lat, lon))
lat, lon

In [ ]:
columns = df.iloc[3, :].values
columns = [col.replace('Variável', 'data') for col in columns]
df.columns = columns

columns

In [ ]:
direction = df.iloc[5:, 15:17].set_index('data')
velocity = df.iloc[5:, 19:21].set_index('data')

In [ ]:
df = pd.concat(
    [direction.dropna(), velocity.dropna()],
    sort=True, axis=1
).dropna()

df.columns = ['direction', 'velocity']
df = df.apply(pd.to_numeric)

kmh_to_ms = (60*60) / 1000
df['velocity'] = df['velocity'] * kmh_to_ms

df.head()

In [ ]:
set(df.index.year), set(df.index.month)

In [ ]:
months = {
    str(month): group for month,
    group in df.groupby(df.index.month)
}

months.keys()

In [ ]:
months['1']

In [ ]:
from windrose import WindroseAxes, WindAxes

ax = WindAxes.from_ax()

dez = '12'
ax, params = ax.pdf(months[dez]['velocity'])

In [ ]:
ax = WindroseAxes.from_ax()

ax.bar(
    months[dez]['direction'],
    months[dez]['velocity'].values,
    normed=True,
    opening=0.8,
    edgecolor='white',
)
ax.set_legend();